# Binaomial Trees (Without Barriers)

In [54]:
# Author : Ashwin Kr.
import numpy as np
# Reformatted Python code in IPython notebook format with a few variable names changed for clarity.

# PARAMETER INITIALIZATION
initial_stock_price = 100  # initial stock price
strike_price = 100         # strike price or exercise price
time_to_maturity = 1       # maturity in years
risk_free_rate = 0.04      # annual risk-free rate
num_steps = 3              # number of time steps in binomial model
up_factor = 1.1            # up-factor in binomial models

# We can take any value of up_factor here but for simplicity, we use 1/up_factor for recombining tree
down_factor = 1 / up_factor  # ensure recombining tree (to return to the same price)
stock_down = initial_stock_price * down_factor  # stock price with up-factor
stock_up = initial_stock_price * up_factor  # stock price with down-factor
option_type = 'C'  # Option Type 'C' means call option, 'P' for put option

In [59]:

# Define the faster binomial tree pricing function
@timing
# Define the binomial pricing model
def binomial_slow(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C'):
    dt = time_to_maturity / num_steps  # Smallest time step


    # Calculating risk-neutral probability using stock prices
    q1 = (initial_stock_price * np.exp(risk_free_rate * dt) - stock_down) / (stock_up - stock_down)
    q2 = (np.exp(risk_free_rate * dt) - down_factor) / (up_factor - down_factor)
    
    print("q1 (using stock prices):", q1)
    print("q2 (using up/down factors):", q2)
    
    # - `q1` is derived using stock prices at the next time step, while
    # - `q2` is the standard risk-neutral probability formula using up/down factors and the risk-free rate.

    disc = np.exp(-risk_free_rate * dt)  # Discount factor

    # Initialize asset prices at maturity (at time step N)
    stock_prices = np.zeros(num_steps + 1)
    stock_prices[0] = initial_stock_price * down_factor**num_steps  # bottom of the tree
    for j in range(1, num_steps + 1):
        stock_prices[j] = stock_prices[j-1] * up_factor / down_factor  # apply up factor
    
    # Initialize option values at maturity
    option_values = np.zeros(num_steps + 1)
    for j in range(0, num_steps + 1):
        if option_type == 'C':  # Call option payoff
            option_values[j] = max(0, stock_prices[j] - strike_price) #you are allowed to buy
        else:  # Put option payoff
            option_values[j] = max(0, strike_price - stock_prices[j]) #you are allowed to sell

    # Backward iteration through tree
    for i in np.arange(num_steps, 0, -1):
        for j in range(0, i):
            option_values[j] = disc * (q2 * option_values[j+1] + (1 - q2) * option_values[j])

    return option_values[0]

# Running the binomial slow function
binomial_slow(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C')


q1 (using stock prices): 0.5464994307846508
q2 (using up/down factors): 0.5464994307846508
Function binomial_slow took 0.0003 seconds


9.09466556261714

In [60]:
# Reformatted Python code with variable name changes and explanation
# Define the faster binomial tree pricing function
@timing
def binomial_fast(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C'):
    # Precompute constants
    dt = time_to_maturity / num_steps  # Smallest time step
    q = (np.exp(risk_free_rate * dt) - down_factor) / (up_factor - down_factor)  # Risk-neutral probability
    disc = np.exp(-risk_free_rate * dt)  # Discount factor

    # Initialise asset prices at maturity - Time step N
    asset_prices = initial_stock_price * down_factor ** np.arange(num_steps, -1, -1) * up_factor ** np.arange(0, num_steps + 1, 1)

    # Initialise option values at maturity
    option_values = np.maximum(asset_prices - strike_price, np.zeros(num_steps + 1))

    # Step backwards through the tree
    for i in np.arange(num_steps, 0, -1):
        option_values = disc * (q * option_values[1:i + 1] + (1 - q) * option_values[0:i])

    return option_values[0]

# Running the binomial_tree_fast function
binomial_fast(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C')


Function binomial_fast took 0.0001 seconds


9.094665562617147

In [62]:
# # Timing decorator function
def timing(f):
    def wrap(*args, **kw):
        start_time = time.time()
        result = f(*args, **kw)
        end_time = time.time()
        print(f'Function {f.__name__} took {(end_time - start_time):.4f} seconds')
        return result
    return wrap

for N in [2,50]:
    print(f"Running for N = {N}...")
    
    binomial_slow(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C')
    binomial_fast(strike_price, time_to_maturity, initial_stock_price, risk_free_rate, num_steps, up_factor, down_factor, option_type='C')
    

Running for N = 2...
q1 (using stock prices): 0.5464994307846508
q2 (using up/down factors): 0.5464994307846508
Function binomial_slow took 0.0002 seconds
Function binomial_fast took 0.0004 seconds
Running for N = 50...
q1 (using stock prices): 0.5464994307846508
q2 (using up/down factors): 0.5464994307846508
Function binomial_slow took 0.0002 seconds
Function binomial_fast took 0.0001 seconds
